In [22]:
%%writefile prefix.cu
#include <iostream>
#include <cstdlib>
#include <ctime>
#include <cuda_runtime.h>

#define BLOCK_SIZE 256

// Kernel for parallel prefix sum
__global__ void prefixSum(int *input, int *output, int n) {
    extern __shared__ int temp[];

    int tid = threadIdx.x;
    int idx = blockIdx.x * blockDim.x + tid;

    // Load input into shared memory
    temp[tid] = (idx < n) ? input[idx] : 0;
    __syncthreads();

    // Reduction phase
    for (int stride = 1; stride < blockDim.x; stride *= 2) {
        int index = 2 * stride * (tid + 1) - 1;
        if (index < blockDim.x) {
            temp[index] += temp[index - stride];
        }
        __syncthreads();
    }

    // Downsweep phase
    for (int stride = blockDim.x / 2; stride > 0; stride /= 2) {
        int index = 2 * stride * (tid + 1) - 1;
        if (index + stride < blockDim.x) {
            temp[index + stride] += temp[index];
        }
        __syncthreads();
    }

    // Store result to output
    if (idx < n) {
        output[idx] = temp[tid];
    }
}

// Wrapper function for inclusive prefix sum calculation
void inclusivePrefixSum(int *d_input, int *d_output, int n) {
    int numBlocks = (n + BLOCK_SIZE - 1) / BLOCK_SIZE;

    // Create CUDA events for timing
    cudaEvent_t start, stop;
    cudaEventCreate(&start);
    cudaEventCreate(&stop);

    // Record start time
    cudaEventRecord(start);

    // Perform parallel prefix sum
    prefixSum<<<numBlocks, BLOCK_SIZE, BLOCK_SIZE * sizeof(int)>>>(d_input, d_output, n);
    cudaDeviceSynchronize();

    // Record stop time
    cudaEventRecord(stop);
    cudaEventSynchronize(stop);

    // Calculate elapsed time
    float milliseconds = 0;
    cudaEventElapsedTime(&milliseconds, start, stop);

   std::cout << "Execution Time: " << milliseconds * 1000<< " microseconds" << std::endl;
}

int main() {
    int n = 5; // Size of the array (assuming 32n elements)
    int arraySize = 32 * n;

    // Host array
    int *h_input = new int[arraySize];

    // Seed for random number generation
    srand(time(nullptr));

    // Generate random numbers for the array
    for (int i = 0; i < arraySize; ++i) {
        h_input[i] = rand() % 100; //
    }

    // Device arrays
    int *d_input, *d_output;
    cudaMalloc(&d_input, arraySize * sizeof(int));
    cudaMalloc(&d_output, arraySize * sizeof(int));

    // Copy data from host to device
    cudaMemcpy(d_input, h_input, arraySize * sizeof(int), cudaMemcpyHostToDevice);

    // Perform inclusive prefix sum
    inclusivePrefixSum(d_input, d_output, arraySize);

    // Copy result back to host
    int *h_output = new int[arraySize];
    cudaMemcpy(h_output, d_output, arraySize * sizeof(int), cudaMemcpyDeviceToHost);

    // Print results
    std::cout << "Input array: ";
    for (int i = 0; i < arraySize; ++i) {
        std::cout << h_input[i] << " ";
    }
    std::cout << std::endl;

    std::cout << "Inclusive Prefix Sum: ";
    for (int i = 0; i < arraySize; ++i) {
        std::cout << h_output[i] << " ";
    }
    std::cout << std::endl;

    // Clean up
    delete[] h_input;
    delete[] h_output;
    cudaFree(d_input);
    cudaFree(d_output);

    return 0;
}


Overwriting prefix.cu


In [24]:
!nvcc -o prefix prefix.cu
!./prefix

Execution Time: 178.176 microseconds
Input array: 55 68 31 25 16 96 33 71 91 41 68 85 63 47 14 89 10 83 56 61 82 46 38 51 1 47 31 90 9 49 38 16 17 21 41 33 69 26 4 12 67 24 49 30 23 15 19 33 99 28 95 81 74 85 85 27 84 68 17 94 17 7 10 86 80 52 71 49 78 27 61 98 3 62 28 26 78 0 12 29 28 59 62 54 44 47 81 29 15 51 75 84 58 85 22 39 89 93 40 20 72 54 18 75 16 98 54 94 50 66 75 30 25 38 84 21 37 18 2 5 69 77 41 27 15 64 18 4 9 59 24 82 13 42 9 81 93 63 28 43 81 3 74 58 93 10 32 31 28 34 88 97 64 29 77 79 45 95 83 55 
Inclusive Prefix Sum: 55 123 154 179 195 291 324 395 486 527 595 680 743 790 804 893 903 986 1042 1103 1185 1231 1269 1320 1321 1368 1399 1489 1498 1547 1585 1601 1618 1639 1680 1713 1782 1808 1812 1824 1891 1915 1964 1994 2017 2032 2051 2084 2183 2211 2306 2387 2461 2546 2631 2658 2742 2810 2827 2921 2938 2945 2955 3041 3121 3173 3244 3293 3371 3398 3459 3557 3560 3622 3650 3676 3754 3754 3766 3795 3823 3882 3944 3998 4042 4089 4170 4199 4214 4265 4340 4424 4482 4567 4589 462

In [25]:
!wget https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64/nsight-systems-2023.2.3_2023.2.3.1001-1_amd64.deb
!apt update
!apt install ./nsight-systems-2023.2.3_2023.2.3.1001-1_amd64.deb
!apt --fix-broken install

--2023-12-21 08:54:55--  https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64/nsight-systems-2023.2.3_2023.2.3.1001-1_amd64.deb
Resolving developer.download.nvidia.com (developer.download.nvidia.com)... 152.199.20.126
Connecting to developer.download.nvidia.com (developer.download.nvidia.com)|152.199.20.126|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 317705436 (303M) [application/x-deb]
Saving to: ‘nsight-systems-2023.2.3_2023.2.3.1001-1_amd64.deb.1’

nsight-systems-2023 100%[===================>] 302.99M   274MB/s    in 1.1s    

2023-12-21 08:54:56 (274 MB/s) - ‘nsight-systems-2023.2.3_2023.2.3.1001-1_amd64.deb.1’ saved [317705436/317705436]

Hit:1 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:2 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:3 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:5 https://cloud.r-project.org/bin/linux/ubu

In [19]:
cuda_code =""" #include <iostream>
#include <cstdlib>
#include <ctime>
#include <cuda_runtime.h>

#define BLOCK_SIZE 256

// Kernel for parallel prefix sum
__global__ void prefixSum(int *input, int *output, int n) {
    extern __shared__ int temp[];

    int tid = threadIdx.x;
    int idx = blockIdx.x * blockDim.x + tid;

    // Load input into shared memory
    temp[tid] = (idx < n) ? input[idx] : 0;
    __syncthreads();

    // Reduction phase
    for (int stride = 1; stride < blockDim.x; stride *= 2) {
        int index = 2 * stride * (tid + 1) - 1;
        if (index < blockDim.x) {
            temp[index] += temp[index - stride];
        }
        __syncthreads();
    }

    // Downsweep phase
    for (int stride = blockDim.x / 2; stride > 0; stride /= 2) {
        int index = 2 * stride * (tid + 1) - 1;
        if (index + stride < blockDim.x) {
            temp[index + stride] += temp[index];
        }
        __syncthreads();
    }

    // Store result to output
    if (idx < n) {
        output[idx] = temp[tid];
    }
}

// Wrapper function for inclusive prefix sum calculation
void inclusivePrefixSum(int *d_input, int *d_output, int n) {
    int numBlocks = (n + BLOCK_SIZE - 1) / BLOCK_SIZE;

    // Create CUDA events for timing
    cudaEvent_t start, stop;
    cudaEventCreate(&start);
    cudaEventCreate(&stop);

    // Record start time
    cudaEventRecord(start);

    // Perform parallel prefix sum
    prefixSum<<<numBlocks, BLOCK_SIZE, BLOCK_SIZE * sizeof(int)>>>(d_input, d_output, n);
    cudaDeviceSynchronize();

    // Record stop time
    cudaEventRecord(stop);
    cudaEventSynchronize(stop);

    // Calculate elapsed time
    float milliseconds = 0;
    cudaEventElapsedTime(&milliseconds, start, stop);

   std::cout << "Execution Time: " << milliseconds * 1000<< " microseconds" << std::endl;
}
int main() {
    int n = 5;
    int arraySize = 32 * n;

    // Host array
    int *h_input = new int[arraySize];

    // Seed for random number generation
    srand(time(nullptr));

    // Generate random numbers for the array
    for (int i = 0; i < arraySize; ++i) {
        h_input[i] = rand() % 100; //
    }

    // Device arrays
    int *d_input, *d_output;
    cudaMalloc(&d_input, arraySize * sizeof(int));
    cudaMalloc(&d_output, arraySize * sizeof(int));

    // Copy data from host to device
    cudaMemcpy(d_input, h_input, arraySize * sizeof(int), cudaMemcpyHostToDevice);

    // Perform inclusive prefix sum
    inclusivePrefixSum(d_input, d_output, arraySize);

    // Copy result back to host
    int *h_output = new int[arraySize];
    cudaMemcpy(h_output, d_output, arraySize * sizeof(int), cudaMemcpyDeviceToHost);

    // Print results
    std::cout << "Input array: ";
    for (int i = 0; i < arraySize; ++i) {
        std::cout << h_input[i] << " ";
    }
    std::cout << std::endl;

    std::cout << "Inclusive Prefix Sum: ";
    for (int i = 0; i < arraySize; ++i) {
        std::cout << h_output[i] << " ";
    }
    std::cout << std::endl;

    // Clean up
    delete[] h_input;
    delete[] h_output;
    cudaFree(d_input);
    cudaFree(d_output);

    return 0;
}

"""
with open('prefix.cu', 'w') as f:
    f.write(cuda_code)

# Compile the CUDA code
!nvcc -o prefix prefix.cu

# Run the executable with Nsight Systems profiling
!nsys profile -t cuda --stats=true ./prefix

Execution Time: 248.736 microseconds
Input array: 68 66 58 12 2 96 42 39 34 34 62 47 85 90 98 66 13 30 14 6 77 8 80 49 99 90 99 79 85 86 82 53 4 40 18 58 89 60 97 23 46 11 23 84 1 73 2 67 3 16 25 32 24 57 33 23 47 32 54 84 70 37 90 74 29 8 32 18 20 82 94 18 45 17 2 99 42 4 66 45 72 91 29 48 48 62 24 47 46 78 32 68 67 74 42 49 82 26 19 54 60 13 72 6 82 27 5 24 83 23 21 8 14 2 56 14 64 80 13 62 11 45 30 78 19 24 79 53 3 99 59 63 12 84 21 47 63 78 23 46 1 45 54 67 47 11 81 64 43 47 26 54 92 9 85 64 33 64 69 88 
Inclusive Prefix Sum: 68 134 192 204 206 302 344 383 417 451 513 560 645 735 833 899 912 942 956 962 1039 1047 1127 1176 1275 1365 1464 1543 1628 1714 1796 1849 1853 1893 1911 1969 2058 2118 2215 2238 2284 2295 2318 2402 2403 2476 2478 2545 2548 2564 2589 2621 2645 2702 2735 2758 2805 2837 2891 2975 3045 3082 3172 3246 3275 3283 3315 3333 3353 3435 3529 3547 3592 3609 3611 3710 3752 3756 3822 3867 3939 4030 4059 4107 4155 4217 4241 4288 4334 4412 4444 4512 4579 4653 4695 4744 4826 